# The Eikonal Equation in 1D with scikit-fmm and DeepXDE

In [ ]:
import skfmm
import numpy as np
import matplotlib.pyplot as plt
import deepxde as dde

## Problem definition and scikit-fmm

In [ ]:
# Define velocity field on interval [-1, 1]
def v(x):
    return 1.0 + 0.0*x

In [ ]:
# Plot velocity field
n = 101
xmin, xmax = -1, 1
x = np.linspace(xmin, xmax, n)
plt.plot(x, v(x))
plt.xlabel('x')
plt.ylabel('v(x)')
plt.title('Velocity field');

In [ ]:
# Define source points at -1 and 1 (at the domain boundary)
phi = np.ones_like(x)
phi[0] = -1
phi[-1] = -1

In [ ]:
# Plot minimum distance to a source point
dx = (xmax - xmin)/(n-1)
d = skfmm.distance(phi, dx)
plt.plot(x, d)
plt.xlabel('x')
plt.ylabel('d(x)')
plt.title('Distance to boundary');

In [ ]:
# Calculate travel time via fast marching method (FMM)
u = skfmm.travel_time(phi, v(x), dx)
plt.plot(x, u)
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('Travel time to boundary');

## PINN solution with DeepXDE

In [ ]:
# Reference travel time for constant velocity
def u_ref(x):
    return 1 - np.abs(x)

In [ ]:
# Generate some noisy observations in the left half of the domain
n_obs = 30
x_obs = np.random.uniform(-1, 0, n_obs)
u_obs = u_ref(x_obs) * np.random.normal(1.0, 0.1, n_obs)
plt.plot(x, u_ref(x), label='true')
plt.scatter(x_obs, u_obs, color='orange', label='observed')
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('Observed travel times');

In [ ]:
# Define computational domain
geom = dde.geometry.Interval(-1.0, 1.0)

In [ ]:
# Define boundary condition for source points at -1 and 1
bc = dde.icbc.DirichletBC(geom, u_ref, lambda _, on_boundary: on_boundary)

In [ ]:
# 1D Eikonal equation
def eikonal(x, y):
    """
    |dy/dx| = 1/v(x)
    """
    du_x = dde.grad.jacobian(y, x)
    xt = x[:,0]
    return du_x**2 - v(xt)**-2

In [ ]:
# Training data
obs = dde.icbc.PointSetBC(x_obs.reshape(-1, 1), u_obs.reshape(-1, 1))

In [ ]:
# Combined DeepXDE data object
data = dde.data.PDE(
    geom,
    eikonal,
    [bc, obs],
    num_domain=100,
    num_boundary=2,
    anchors=x_obs.reshape(-1, 1),
    solution=u_ref
)

In [ ]:
# Neural network
net = dde.nn.FNN([1] + [10] * 3 + [1], "tanh", "Glorot uniform")
model = dde.Model(data, net)

In [ ]:
# Compile DeepXDE model
model.compile("adam", lr=0.001)

In [ ]:
# Train model
losshistory, train_state = model.train(iterations=50000)

In [ ]:
# Plot results
dde.saveplot(losshistory, train_state, issave=False, isplot=True)